In [2]:
#Modules to install via pip pandas,ipynb
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import json
from pprint import pprint
import os
import import_ipynb
import sys

sys.path.append('../')
from functions import *
from trace_analysis import *
from pandas.plotting import scatter_matrix



In [13]:
directory=os.getcwd()
traces=directory+"/traces"
#+"/cooja-9nodes/"
#print(directory)

for directory in os.listdir(traces):
        
        #with open(dir+"/" + file, 'r') as f:
        
        for file in os.listdir(traces+"/"+directory):
            #print("Importing "+ file)
            with open(dir+"/" + file, 'r') as f:
                



TypeError: unsupported operand type(s) for +: 'builtin_function_or_method' and 'str'